In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine

In [2]:
engine = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
pgn = open("../data/RuyLopezMarshall.pgn")

In [4]:
games_id = []
games = []

while True:
    offset = pgn.tell()
    headers = chess.pgn.read_headers(pgn)
    if headers is None:
        break
    games_id.append(offset)

for offset in games_id:
    pgn.seek(offset)
    games.append(chess.pgn.read_game(pgn))

In [5]:
dados = []
count = -1

for game in games:
    count += 1
    row = {}
    head = game.headers

    if head['Site'].endswith(' INT') or head['Site'] == '?':
        continue

    row['id'] = games_id[count]
    row['Event'] = head['Event']
    row['Site'] = head['Site']
    row['Date'] = head['Date'].split('.')[0]
    row['White'] = head['White']
    row['Black'] = head['Black']
    row['Result'] = head['Result']
    row['BlackElo'] = head['BlackElo']
    row['ECO'] = 'C89'
    row['WhiteElo'] = head['WhiteElo']

    #quem ganhou
    if head['Result'] == '0-1':
        row['WhiteWon'] = 0
        row['BlackWon'] = 1
    elif head['Result'] == '1-0':
        row['WhiteWon'] = 1
        row['BlackWon'] = 0
    else:
        row['WhiteWon'] = 0
        row['BlackWon'] = 0

    #qntd de moves
    final_position = game[0].end().board()
    row['Moves'] = final_position.fullmove_number

    #qntd de peças restantes
    final_fen = final_position.board_fen()
    white_count = 0
    black_count = 0
    white_pieces = ['P','N','B','R','Q']
    black_pieces = ['p','n','b','r','q']
    white_has_queen = 0
    black_has_queen = 0
    for char in range(0, len(final_fen)):
        if final_fen[char] in white_pieces:
            white_count += 1
        elif final_fen[char] in black_pieces:
            black_count += 1

        if final_fen[char] == 'Q':
            white_has_queen = 1
        elif final_fen[char] == 'q':
            black_has_queen = 1
        
    
    row['WhiteFinalPiecesCount'] = white_count
    row['BlackFinalPiecesCount'] = black_count
    row['FinalPiecesCount'] = white_count + black_count
    if white_has_queen == 0 and black_has_queen == 0:
        row['QueenTrade'] = 1
    else:
        row['QueenTrade'] = 0

        
    dados.append(row)  
     
df = pd.DataFrame(dados)

In [6]:
#corrigir places/Sites
def string_analyzer(row):
    s = row['Site']
    splitted_s = s.split(' ')
    first_part = splitted_s[:-1]
    last_part = splitted_s[-1]
    city = ''

    if last_part.isupper() and len(splitted_s) > 1:
        for w in first_part:
            city += w + ' '
    else:
        city = s

    row['Site'] = city

    return row

df = df.apply(lambda row: string_analyzer(row=row), axis=1)

# testes

In [7]:
position = games[1000][0].end().board()
#qntd de peças restantes
fen = position.board_fen()
fen

'3Q1rk1/8/p2b2q1/1p1p3p/2PPb3/1P2RpP1/P2B1P1N/6K1'

In [8]:
print(position.outcome())

None


In [9]:
print(dados[0])

{'id': 0, 'Event': 'New York', 'Site': 'New York', 'Date': '1918', 'White': 'Morrison, John Stuart', 'Black': 'Marshall, Frank James', 'Result': '0-1', 'BlackElo': '', 'ECO': 'C89', 'WhiteElo': '', 'WhiteWon': 0, 'BlackWon': 1, 'Moves': 85, 'WhiteFinalPiecesCount': 1, 'BlackFinalPiecesCount': 3, 'FinalPiecesCount': 4, 'QueenTrade': 0}


In [10]:
#df.to_csv('../data/df_jogos.csv', index=False)
df

,id,Event,Site,Date,White,Black,Result,BlackElo,ECO,WhiteElo,WhiteWon,BlackWon,Moves,WhiteFinalPiecesCount,BlackFinalPiecesCount,FinalPiecesCount,QueenTrade
0,0,New York,New York,1918,"Morrison, John Stuart","Marshall, Frank James",0-1,,C89,,0,1,85,1,3,4,0
1,1168,New York,New York,1918,"Capablanca, Jose Raul","Marshall, Frank James",1-0,,C89,,1,0,36,7,5,12,0
2,1772,Brno,Brno,1921,"Sery, S.","Vecsey, Zoltan",1-0,,C89,,1,0,17,12,10,22,0
3,2129,NLD-ch,Nijmegen,1921,"Te Kolste, Jan Willem","Loman, Rudolf",0-1,,C89,,0,1,19,11,10,21,0
4,2522,USA sim,USA,1926,"Lasker, Emanuel","Bigelow, Horace Ransom",0-1,,C89,,0,1,20,11,11,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979,2113381,23rd European Teams,Terme Catez,2021,"Agopov,M","Ragger,M",1/2-1/2,2668,C89,2430,0,0,55,2,1,3,1
2980,2117758,8th Sunway Sitges 2021,Sitges,2021,"Kurmann,O","Van Dael,Siem",1-0,2300,C89,2466,1,0,51,8,5,13,0
2981,2118961,7th Gashimov Mem Blitz,Baku,2021,"Karjakin,Sergey","Caruana,F",1/2-1/2,2792,C89,2743,0,0,37,4,4,8,1
2982,2119588,World Rapid Women 2021,Warsaw,2021,"Loskutova,Viktoriya","Abdumalik,Z",1-0,2491,C89,2100,1,0,42,6,5,11,1
